<div style="
    background-color:rgb(210, 230, 255);
    color:rgb(0, 0, 0);
    padding: 10px;
    border-radius: 10px;
    font-weight: bold;">
<h3>Modèle d'apprentissage automatique pour la prédiction d’usage le plus probable d'une friche en France</h3>
<ul>
    <li>Dans le cadre du projet PROFIL : https://github.com/heuzef/cartofriches</li>
    <li>Auteurs : Heuzef (https://heuzef.com) ; Frédéric Vincent (frederic.vincent@protonmail.com)</li>
    <li>Juin 2025</li>
    <li>Destiné au CEREMA</li>
    <li>Les données exploitées sont issues data.gouv.fr, respectant le Standard Friches (version 2022 - rev. v2023-12)</li>
</div>

# Import des librairies

In [2]:
!pip install -r requirements.txt

In [3]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jinja2
import re

# Chargement des données

In [4]:
# Chargement des données silver

df_friches_silver = pd.read_csv("./data/friches_silver.csv", sep=",", low_memory=False)

print("Données des sites référencés dans Cartofriches disponible sur data.gouv.fr (respectant le standard CNIG) :")
print(f"{df_friches_silver.shape[0]} friches chargées — {df_friches_silver.shape[1]} variables")
print("Pas de doublon detectés")

display(df_friches_silver.head())

Données des sites référencés dans Cartofriches disponible sur data.gouv.fr (respectant le standard CNIG) :
2331 friches chargées — 26 variables
Pas de doublon detectés


,site_type,site_securite,site_occupation,site_reconv_type,bati_type,bati_nombre,bati_pollution,bati_vacance,bati_etat,proprio_personne,...,uzf_activite_agricole,uzf_activite_economique,uzf_autres,uzf_equipements_public,uzf_espace_naturel,uzf_espace_remarquable,uzf_habitat,uzf_loisirs_et_tourisme,uzf_mixte_habitat_activite,uzf_sans_objet_ou_non_encore_definie_dans_le_reglement
0,friche d'habitat,0,NaN,0,NaN,2,NaN,NaN,NaN,personne morale,...,0,0,1,0,0,0,0,0,0,0
1,friche d'habitat,0,NaN,0,NaN,2,NaN,NaN,dégradation moyenne,personne physique,...,0,0,1,0,0,0,0,0,0,0
2,friche agro-industrielle,0,NaN,1,NaN,0,NaN,NaN,NaN,personne physique,...,0,0,1,0,0,0,0,0,0,0
3,friche d'habitat,0,NaN,2,NaN,3,NaN,NaN,dégradation moyenne,personne physique,...,0,0,1,0,0,0,0,0,0,0
4,friche agro-industrielle,0,NaN,1,NaN,0,NaN,NaN,NaN,personne physique,...,1,0,0,0,0,0,0,0,0,0


# Extraction des données numériques

In [5]:
df_friches_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 26 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   site_type                                               1997 non-null   object 
 1   site_securite                                           2331 non-null   int64  
 2   site_occupation                                         439 non-null    object 
 3   site_reconv_type                                        2331 non-null   int64  
 4   bati_type                                               433 non-null    object 
 5   bati_nombre                                             2331 non-null   int64  
 6   bati_pollution                                          475 non-null    object 
 7   bati_vacance                                            430 non-null    object 
 8   bati_etat                             

In [6]:
df_friches_silver_numeriques = df_friches_silver.select_dtypes(exclude="object")

In [7]:
df_friches_silver_numeriques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 16 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   site_securite                                           2331 non-null   int64  
 1   site_reconv_type                                        2331 non-null   int64  
 2   bati_nombre                                             2331 non-null   int64  
 3   unite_fonciere_surface                                  2331 non-null   int64  
 4   longitude                                               2331 non-null   float64
 5   latitude                                                2331 non-null   float64
 6   uzf_activite_agricole                                   2331 non-null   int64  
 7   uzf_activite_economique                                 2331 non-null   int64  
 8   uzf_autres                            